In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# DOWNLOAD LINKS
import pandas as pd
link_train = 'https://drive.google.com/file/d/1JlCy6JQ5y6gBcMooJdMaWwc8XeMVIf52/view?usp=sharing'

download = drive.CreateFile({'id': '1JlCy6JQ5y6gBcMooJdMaWwc8XeMVIf52'})
download.GetContentFile('train.csv')

df_train = pd.read_csv('train.csv')

In [ ]:
link_test = 'https://drive.google.com/file/d/1OrC0aelyv_Av90a5-PrmbaEquPRDmYP1/view?usp=sharing'


download_test = drive.CreateFile({'id': '1OrC0aelyv_Av90a5-PrmbaEquPRDmYP1'})
download_test.GetContentFile('test.csv')

df_test = pd.read_csv('test.csv')

In [ ]:
X_train = df_train.text
y_train = df_train.label

X_test = df_test.text
y_test = df_test.label

In [ ]:
print(y_test)

0          video
1          dunya
2          video
3          video
4       yazarlar
          ...   
1995        spor
1996       dunya
1997     turkiye
1998    yazarlar
1999        spor
Name: label, Length: 2000, dtype: object


In [ ]:
#ONE HOT ENCODING

import numpy 
y_train_num = []
for label in y_train:
    if label == 'spor':
        y_train_num.append([0,0,0,0,1])
    elif label == 'dunya':
        y_train_num.append([0,0,0,1,0])
    elif label == 'yazarlar':
        y_train_num.append([0,0,1,0,0])
    elif label == 'turkiye':
        y_train_num.append([0,1,0,0,0])
    elif label == 'video':
        y_train_num.append([1,0,0,0,0])
    else:
        y_train_num.append([1,0,0,0,0])
        print("LABEL IS NOT ONE OF THE FIVE !!") 

y_test_num = []
for label in y_test:
    if label == 'spor':
        y_test_num.append([0,0,0,0,1])
    elif label == 'dunya':
        y_test_num.append([0,0,0,1,0])
    elif label == 'yazarlar':
        y_test_num.append([0,0,1,0,0])
    elif label == 'turkiye':
        y_test_num.append([0,1,0,0,0])
    elif label == 'video':
        y_test_num.append([1,0,0,0,0])
    else:
        y_test_num.append([1,0,0,0,0])
        print("LABEL IS NOT ONE OF THE FIVE !!")



In [ ]:
np_y_train_num = numpy.array(y_train_num)
np_y_test_num = numpy.array(y_test_num)
print ("Train : ",np_y_train_num)
print(y_train)
print("\ntest: ", np_y_test_num)
print(y_test)


Train :  [[0 0 0 0 1]
 [0 0 1 0 0]
 [1 0 0 0 0]
 ...
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]]
0           spor
1       yazarlar
2          video
3          video
4           spor
          ...   
7995        spor
7996        spor
7997       dunya
7998       dunya
7999       dunya
Name: label, Length: 8000, dtype: object

test:  [[1 0 0 0 0]
 [0 0 0 1 0]
 [1 0 0 0 0]
 ...
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]]
0          video
1          dunya
2          video
3          video
4       yazarlar
          ...   
1995        spor
1996       dunya
1997     turkiye
1998    yazarlar
1999        spor
Name: label, Length: 2000, dtype: object


In [ ]:
"""import string
#removing punctuation from the list
X_train = [''.join(c for c in s if c not in string.punctuation) for s in X_train]
df_train['text'] = X_train
#print(df_train['text'])
"""

"import string\n#removing punctuation from the list\nX_train = [''.join(c for c in s if c not in string.punctuation) for s in X_train]\ndf_train['text'] = X_train\n#print(df_train['text'])\n"

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Importing libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [ ]:
#Maximum lenght of a document
maxlen=-1
corp = X_train
for doc in corp:
    tokens=nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen=len(tokens)
print("The maximum number of words in any document is : ",maxlen)

The maximum number of words in any document is :  7974


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)

X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
vocab_size = len(tokenizer.word_index) +1 
print("Vocab size is:", vocab_size)


Vocab size is: 206883


In [ ]:
#Padding using padding = 'pre'

from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen = maxlen) 
X_test = pad_sequences(X_test, maxlen=maxlen)

In [ ]:
# Building the CNN Model
model = Sequential()      # initilaizing the Sequential nature for CNN model
# Adding the embedding layer which will take in maximum of vocab_size words as input and provide a 50 dimensional output o

model.add(Embedding(vocab_size, 50, input_length=maxlen))
model.add(Conv1D(32, 5, activation='softmax'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#other loss option is sparse_categorical_crossentropy. Others do not fit.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7974, 50)          10344150  
_________________________________________________________________
conv1d (Conv1D)              (None, 7970, 32)          8032      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3985, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 127520)            0         
_________________________________________________________________
dense (Dense)                (None, 250)               31880250  
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1255      
Total params: 42,233,687
Trainable params: 42,233,687
Non-trainable params: 0
____________________________________________

In [ ]:
model.fit(X_train, np_y_train_num, validation_data=(X_test, np_y_test_num), epochs=12, batch_size=128, verbose=1)


Epoch 1/12
63/63 [==============================] - 35s 551ms/step - loss: 0.7362 - acc: 0.7860 - val_loss: 0.9777 - val_acc: 0.6525
Epoch 2/12
63/63 [==============================] - 35s 550ms/step - loss: 0.6672 - acc: 0.7900 - val_loss: 0.9522 - val_acc: 0.6545
Epoch 3/12
63/63 [==============================] - 35s 550ms/step - loss: 0.6134 - acc: 0.7962 - val_loss: 0.9408 - val_acc: 0.6535
Epoch 4/12
63/63 [==============================] - 35s 550ms/step - loss: 0.5718 - acc: 0.7950 - val_loss: 0.9271 - val_acc: 0.6515
Epoch 5/12
63/63 [==============================] - 34s 549ms/step - loss: 0.5374 - acc: 0.7934 - val_loss: 0.9185 - val_acc: 0.6575
Epoch 6/12
63/63 [==============================] - 34s 548ms/step - loss: 0.5093 - acc: 0.7955 - val_loss: 0.9155 - val_acc: 0.6585
Epoch 7/12
63/63 [==============================] - 34s 548ms/step - loss: 0.4860 - acc: 0.8075 - val_loss: 0.9078 - val_acc: 0.6600
Epoch 8/12
63/63 [==============================] - 34s 548ms/step - 

In [ ]:
# Getting score metrics from our model
scores = model.evaluate(X_test, np_y_test_num, verbose=0)
# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))

#Scores[0] is loss, Scores[1] is accuracy 
print(scores)
print(model.metrics_names)

Accuracy: 63.05%
[0.91204833984375, 0.6305000185966492]
['loss', 'acc']


In [ ]:
#Using pre-trained word embeddings

from gensim import models
link = 'https://drive.google.com/file/d/10I4dwHSiLZoShooCw4i68t3bFPYMIRqL/view?usp=sharing'
download = drive.CreateFile({'id': '10I4dwHSiLZoShooCw4i68t3bFPYMIRqL'})
download.GetContentFile('cs445_trmodel_p3')
word2vec = models.KeyedVectors.load_word2vec_format('cs445_trmodel_p3', binary=True)


In [ ]:
#print(word2vec.most_similar(positive=["kral","kadın"],negative=["erkek"]))

In [ ]:
# load embedding of gensim into keras model
embedding = word2vec.wv.get_keras_embedding(train_embeddings=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
model = Sequential()
from keras.layers import GlobalMaxPooling1D
model.add(embedding)
model.add(Conv1D(32, 5, activation='softmax'))
#model.add(MaxPooling1D())
model.add(GlobalMaxPooling1D())
model.add(Dense(250, activation='softmax'))
model.add(Dense(5, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 400)         164982800 
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 32)          64032     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 250)               8250      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 1255      
Total params: 165,056,337
Trainable params: 73,537
Non-trainable params: 164,982,800
_________________________________________________________________
None


In [ ]:
# fit the model
model.fit(X_train, np_y_train_num, validation_data=(X_test, np_y_test_num), epochs=12, batch_size=128, verbose=1)

Epoch 1/12
63/63 [==============================] - 35s 553ms/step - loss: 1.6075 - acc: 0.2276 - val_loss: 1.5953 - val_acc: 0.3110
Epoch 2/12
63/63 [==============================] - 35s 550ms/step - loss: 1.5859 - acc: 0.3421 - val_loss: 1.5525 - val_acc: 0.4440
Epoch 3/12
63/63 [==============================] - 35s 550ms/step - loss: 1.5363 - acc: 0.4572 - val_loss: 1.4880 - val_acc: 0.4685
Epoch 4/12
63/63 [==============================] - 35s 550ms/step - loss: 1.4595 - acc: 0.5251 - val_loss: 1.4083 - val_acc: 0.5895
Epoch 5/12
63/63 [==============================] - 35s 550ms/step - loss: 1.3634 - acc: 0.6414 - val_loss: 1.3143 - val_acc: 0.6200
Epoch 6/12
63/63 [==============================] - 35s 550ms/step - loss: 1.2327 - acc: 0.6846 - val_loss: 1.2135 - val_acc: 0.6365
Epoch 7/12
63/63 [==============================] - 35s 550ms/step - loss: 1.1126 - acc: 0.7213 - val_loss: 1.1431 - val_acc: 0.6520
Epoch 8/12
63/63 [==============================] - 35s 550ms/step - 